# Introduction 

## Objective

- Answer the question:
  * For any given day and place is the relationship between observations recorded as "head counts" and those recorded as "sweeps"?
- Document the process of finding the answer so that the exact steps are replicable and clearly demonstrated here.

## About the Data

!["Perfectly straight line in graph comparing supposedly independent variables in Head Count and Sweeps workbook, in Excel Online"](head-count-sweeps-graph-excel-online.png)

I'm troubleshooting a spreadsheet document that was prepared by someone else. Whoever prepared it probably handed it off to someone else before I came into contact. The information in the document looks like it's been copied from other documents which I'm not certain I have access to.

In this document, there's a graph that demonstrates the goal of the document: to compare the relationship between two observation methods. Unfortunately, the graph shows a suspicious degree of idealness: a perfect one-to-one ratio across the entire domain.

My assignment is to trace the error and correct the graph so that it displays the precise ratios calculated from appropriate samples. The samples available for this purpose are counts of aphids collected by the two methods described in the [Objective].

[Objective]: #Objective

### Explore Worksheets

Before getting hands on with _pandas_, I'll open the document in Excel and record what my senses tell me.

#### Unbelievable Graph

- There's a graph in a worksheet called "head counts vs sweeps graphs" which demonstrates the analytical problem encountered/developed by someone else.
- The data supposedly being compared in the graph is cannot be the data that was intended for comparison because the ratio depicted is perfectly linear even though it's comparing real world samples.

#### Work Not Shown

It's clear that the Excel workbook has the results of many calculations, yet there are no spreadsheet formulas in any cells. Spreadsheet formulas would have greatly expedited the verification of the calculations; without any reference to how the calculations were performed and on what values, I'll need to replicate them from scratch.

#### Metadata and Document History

- Data may have been copied from multiple, unidentified sources.
- It's not clear which data is "original" and which is duplicated, amongst the worksheets in the workbook.
- Some data appears to have been summed and then mixed back in with the rest of the data.
- There are no notes from editors of the workbook, and the editors are unidentifiable.

### Summary of Issues

- No metadata
- No convention for categorical label values
- Mixed aggregation levels (some sums mixed with non-summed data)
- Graph plot is based on incorrect values

## Analytical Goals

### Tidy & Plot Ratios

- Data is clean and indexed well enough to align quantitative samples from the two field collection methods.
- Upon plotting the ratios of dependent variables corresponding with each collection method, the graph should not depict an absolutely perfect fit to a line. That is to say, there should be some variation.

### Retain for Reference

- For posterity, the data should be easy to align with any similar data. This makes it possible to identify whether the same data exists in any other file.

# Pre-analysis

## Setup Computing Environment

### Import Python Packages

In [1]:
import re

import pandas
from markdown import markdown
from IPython.display import HTML

## Load Worksheets into _pandas_

I'll make a dictionary of names and data frames, so I can examine the file overall.

In [2]:
data_file = pandas.ExcelFile('../data/2016 combination.xlsx')
sheets = {
    sheet_name: data_file.parse(sheet_name)
    for sheet_name in data_file.sheet_names
}
sheet_names = sheets.keys()
sheet_frames = sheets.values()

### Classify Worksheets

Because I can't trust the accuracy of the data used by the plot in Excel, I need to look at all the sheets and determine the most complete and unadulterated data sets. I'll determine which data belongs to each category, and compare the sets.

#### Compare Columns

To begin to discern which data is original, I'll list the first eight columns of each worksheet, which will provide me a sense for the difference in data structures:

In [3]:
pandas.DataFrame(
    data=[frame.columns for frame in sheet_frames],
    index=pandas.Index(data=sheet_names),
).loc[:, :8]

,0,1,2,3,4,5,6,7,8
Head Counts,Site,Crop,Date,Field,Zadoks_stage,Tiller,EGA_head,EGA_leaf,BCO_head
Sheet2,ID,Province,Collection_Date,Sample_by_week,Date_by_week,Date,Julian_date,Site,Field_name
Sweep Samples Cereals,ID,Province,Collection_Date,Sample_by_week,Date_by_week,Date,Julian_date,Site,Field_name
Head Counts Edited,Date,Site,Crop,Field,Sample Type,Unnamed: 5,Zadoks_stage,Tiller,EGA_alate
Sweep Samples Cereals Edited,Date,Site,Crop,Field_name,Sample Type,Total Sweeps,Unnamed: 6,Unnamed: 7,EGA_alate
Data Sheets Combined,ID,Date,Site,Crop,Field_name,Sample Type,Total Sweeps,Zadoks_stage,Tiller
Pivot chart LH phen,None,None,None,None,None,None,None,None,None
leafhoppers 2016 cereal sweeps,Collection_Date,Sample_by_week,Date_by_week,Date,Julian_date,Site,Field_name,Crop,Distance(m)
Sheet3,None,None,None,None,None,None,None,None,None
aphid sweep vs head count,ID,Date,Site,Crop,Field_name,Sample Type,Total Sweeps,Zadoks_stage,Tiller


Based on sheet names, column names, and similarities between columns sets, I can probably group the sheets like so:

#### Tiller Head Count

- **Head Counts**
- **Head Counts Edited**

#### Net Sweep

- **Sheet2**
- **Sweep Samples Cereals**
- **Sweep Samples Cereals Edited**
- **leafhoppers 2016 cereal sweeps**

#### Extraneous

- United:
  - **Data Sheets Combined**
- Analytical experiments:
  - **Pivot chart LH phen**
  - **Sheet3**
  - **aphid sweep vs head count**
  - **head counts vs sweeps graphs**

I don't want any of the sheets grouped under "extraneous". The **Data Sheets Combined** sheet is uninteresting to me because I want to "combine" the sheets in my own way, rather than trust duplicated spreadsheet ranges.

## Tidy Up

### Normalize Column Names

I've already noticed at least one leading space on a column name ("spiders"), and variations in capitalization. To compensate for this, I'll strip all leading and trailing whitespace from the column names.

To avoid confusion due to variations in capitalization, I'll convert all column names to lower case.

In [4]:
for sheet in sheets.values():
    if sheet.columns.size > 0:
        sheet.columns = sheet.columns.str.strip().str.lower()

### Convert Date & Time Format

Before I can easily examine dates from the worksheets, I must convert them to proper date values for Python and *pandas*. I'll check the date formats:

In [5]:
(
    pandas.concat(
        {
            name: sheet.loc[
                :,
                sheet.columns.str.contains('date')
            ].reset_index(drop=True)
            for (name, sheet) in sheets.items()
                if sheet.columns.size > 0
        },
        axis='columns',
    )
    .loc[0]
    .unstack()
    .fillna('')
)

,collection_date,date,date_by_week,julian_date
Data Sheets Combined,,2016-08-12 00:00:00,,
Head Counts,,04/08/2016,,
Head Counts Edited,,2016-08-04 00:00:00,,
Sheet2,12_08_2016,Aug_12,0,0
Sweep Samples Cereals,12_08_2016,Aug_12,0,0
Sweep Samples Cereals Edited,,2016-08-12 00:00:00,,
aphid sweep vs head count,,2016-08-04 00:00:00,,
leafhoppers 2016 cereal sweeps,12_08_2016,Aug_12,0,0


According to the documentation for [`pandas.to_datetime`], I can convert a string to a true datetime value. The function uses [Python datetime format strings]. Here are the date columns, the format strings, and the worksheet names I see:

***collection_date*** (`%m_%d_%Y`):

- **leafhoppers 2016 cereal sweeps**
- **Sheet2**
- **Sweep Samples Cereals**

***date*** (`%m/%d/%Y`):

- **Head Counts**

***date*** (no conversion):

- **aphid sweep vs head count**
- **Sweep Samples Cereals Edited**
- **Head Counts Edited**
- **Data Sheets Combined**

In some worksheets, the ***date*** column is present even though there's a better column for the date (***collection_date***). I'll drop the ***date*** column and rename the ***collection_date*** column to fill its place.

After conversion, I'll output a sample date from the column to prove that the column ***date*** is in fact a datetime.

[`pandas.to_datetime`]: http://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.to_datetime.html
[Python datetime format strings]: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [6]:
for name, sheet in ((name, sheets[name]) for name in (
    'Head Counts',
    'Sheet2',
    'Sweep Samples Cereals',
    'leafhoppers 2016 cereal sweeps',
)):
    display(HTML(markdown(f'#### {name}')))
    if name == 'Head Counts':
        sheet['date'] = pandas.to_datetime(
            sheet['date'],
            format='%d/%m/%Y',
        )
    else:
        sheet.drop(columns='date', inplace=True)
        sheet.rename(
            columns={'collection_date': 'date'},
            inplace=True,
        )
        sheet['date'] = pandas.to_datetime(
            sheet['date'],
            format='%d_%m_%Y',
        )
    display(sheet.date.head(1))

0   2016-08-04
Name: date, dtype: datetime64[ns]

0   2016-08-12
Name: date, dtype: datetime64[ns]

0   2016-08-12
Name: date, dtype: datetime64[ns]

0   2016-08-12
Name: date, dtype: datetime64[ns]

# Select Primary Data Sets

## Sweep Samples

### Candidates

- **Sheet2**
- **Sweep Samples Cereals**
- **Sweep Samples Cereals Edited**
- **leafhoppers 2016 cereal sweeps**

From my hands-on exploration in Excel, I got the feeling that **Sheet2** is the most complete. It generally had more rows and looked tidiest. I'll compare it to the other candidates, one at a time.

### **Sheet2** vs **Sweep Samples Cereals**

I'd like to confirm my hope that all data in **Sweep Samples Cereals** is included in the vaguely named **Sheet2**. I'll put them both in a dictionary called **compare_sheets** for quick reference.

In [7]:
sheet_names = (
    'Sweep Samples Cereals',
    'Sheet2',
)
sheets_to_compare = {
    name: sheets[name]
    for name in sheet_names
}

I'll refer to **Sweep Sample Cereals** as **ssc** and I'll continue to refer to **Sheet2** as **s2**. 

In [8]:
ssc, s2 = sheets_to_compare.values()

#### Columns

I'll convert each column list to a `Series` and display them side-by-side in a `DataFrame`.

In [9]:
from typing import Dict


def emojify_boolean(frame: pandas.DataFrame) -> pandas.DataFrame:
    return (
        frame.replace({
            True: '✅',
            False: '✖️',
        })
    )


def compare_columns(sheets_to_compare: Dict[str, pandas.DataFrame]) -> pandas.DataFrame:
    return emojify_boolean(
        ~pandas.DataFrame(
            data={
                name: sheet.columns.to_series(name=name)
                for name, sheet in sheets_to_compare.items()
            }
        ).isna()
    )

In [10]:
compare_columns(sheets_to_compare)

,Sweep Samples Cereals,Sheet2
( damsel bug)nabis_americoferus_adult,✅,✅
1st_instar_ega,✅,✅
1st_instar_macrosteles,✅,✅
2nd_instar_ega,✅,✅
2nd_instar_macrosteles,✅,✅
3rd_instar_ega,✅,✅
3rd_instar_ega_pre-alate,✅,✅
3rd_instar_macrosteles,✅,✅
4th_instar_macrosteles,✅,✅
4th_instar_pre-alate,✅,✅


I see only a few differences. I'll determine the exact asymmetry using *pandas*'s `Index.symmetric_difference` (which is available on the `columns` property, since columns are technically also an `Index`):

In [11]:
ssc.columns.symmetric_difference(s2.columns).tolist()

['ega/20 sweeps',
 'ega/sweep',
 'total ega',
 'total sweeps',
 'unnamed: 129',
 'unnamed: 133']

I believe the unnamed columns are empty. The other columns have calculated values that I don't trust. **Sweep Samples Cereals** is probably useless for my purposes.

#### Date Range

In [13]:
def len_unique(pandas_object):
    return len(pandas_object.unique())


pandas.DataFrame(
    {
        name: frame.date
        for name, frame in sheets_to_compare.items()
    }
).apply(
    [
        pandas.Series.max,
        pandas.Series.min,
        len_unique,
    ]
)

,Sweep Samples Cereals,Sheet2
max,2017-07-08 00:00:00,2017-07-08 00:00:00
min,2016-06-05 00:00:00,2016-06-05 00:00:00
len_unique,33,34


I need to check if *all* the dates in **Sweep Samples Cereals** are in **Sheet2**.

In [14]:
ssc.date.isin(s2.date).all()

True

I'm curious if the converse is also true:

In [15]:
s2.date.isin(ssc.date).all()

False

Since **s2** has some dates which **ssc** does not, I'd like to know how big the difference is. I'll view all date values in **s2** which fail to match with those in **ssc**, ignoring duplicates.

In [16]:
(
    s2.date.loc[~s2.date.isin(ssc.date)]
    .drop_duplicates()
)

486   2017-06-17
568   2016-06-06
Name: date, dtype: datetime64[ns]

So, this isn't a lot, but it's worth noting. These dates are only in **s2**.

#### Possible need to merge two sheets; avoiding reduced data (totals)

I see a ***total sweeps*** column only in **Sweep Samples Cereals**, which leads me to suspect that **Sweep Samples Cereals** comprises aggregates (sums), possibly based on values present in **Sheet2**. However, **Sheet2** has two additional dates, which means that it has data that the other worksheet doesn't. Now, it's a matter of determining whether **Sheet2** is comprehensive enough to preclude the use of **Sweep Samples Cereals**.

##### Value Counts on ***distance(m)***

It's my impression that the actual, physical sweeps were at different distances from some physical origin, and later reduced to sums. If I peek at the ***distance(m)*** column, I should see a clear difference between these varied distances and the "distance" value used by records which have multiple sweeps combined. (Ideally, there wouldn't be a distance value for aggregates at all, but this illustrates the problem of mixing raw data with partially analysed data.)

In [18]:
display(
    pandas.concat(
        {
            name: frame['distance(m)']
            for name, frame in sheets_to_compare.items()
        },
        axis='columns',
    )
    .apply(pandas.value_counts)
    .fillna('-')
)

,Sheet2,Sweep Samples Cereals
5,105,-
10,107,-
25,106,-
50,109,-
100,93,-
Combined,92,92
0,56,-


Indeed, **Sheet2** has data from observations at various "distances", while **Sweep Samples Cereals** has only the label, ***Combined***.

**Sheet2** has more rows because it isn't totalling up the sweeps from various distances. That makes **Sheet2** less processed, and therefore more "raw" (less likely to be corrupted by human error during prior analysis).
Since I don't want reduced (summed) data, I don't want **Sweep Samples Cereals**.

#### Conclusion

Most optimal of these two:

- **Sheet2**

### **Sheet2** vs **Sweep Samples Cereals Edited**

In [19]:
sheets_to_compare = {
    name: sheets[name]
    for name in (
        'Sweep Samples Cereals Edited', 
        'Sheet2',
    )
}
ssce, s2 = sheets_to_compare.values()
sheet_names = sheets_to_compare.keys()

#### Columns

In [20]:
pandas.options.display.max_rows = 140

compare_columns(sheets_to_compare)

,Sweep Samples Cereals Edited,Sheet2
( damsel bug)nabis_americoferus_adult,✅,✅
1st_instar_ega,✖️,✅
1st_instar_macrosteles,✖️,✅
2nd_instar_ega,✖️,✅
2nd_instar_macrosteles,✖️,✅
3rd_instar_ega,✖️,✅
3rd_instar_ega_pre-alate,✖️,✅
3rd_instar_macrosteles,✖️,✅
4th_instar_macrosteles,✖️,✅
4th_instar_pre-alate,✖️,✅


**Sweep Samples Cereals Edited** seems to have left out some columns that would be expected to carry finely categorized subjects, such as various instars of aphids. Furthermore, it appears that **Sweep Samples Cereals Edited** has a column, ***Total Sweeps***, which is probably an artefact from previous analysis efforts. It's also missing the ***Distance(m)*** column; another sign of analysis, and therefore loss of some information.

Unless analysis of date labels reveals that **Sweep Samples Cereals Edited** has dates that are missing from **Sheet2**, I'll assume it's not worth closer examination.

#### Date Range

In [21]:
def len_unique(pandas_object):
    return len(pandas_object.unique())


pandas.DataFrame(
    {
        name: frame.date
        for name, frame in sheets_to_compare.items()
    }
).apply([
    pandas.Series.max,
    pandas.Series.min,
    len_unique,
    len,
])

,Sweep Samples Cereals Edited,Sheet2
max,2016-08-15 00:00:00,2017-07-08 00:00:00
min,2016-06-05 00:00:00,2016-06-05 00:00:00
len_unique,32,34
len,668,668


It appears that **Sweep Samples Cereals Edited**, like **Sweep Samples Cereals**, has slightly fewer unique dates than **Sheet2** but the same range, so I probably won't miss anything if I ignore it. To be sure, I need to check if all those specific dates in **Sweep Samples Cereals Edited** are actually in **Sheet2**, and not just the range.

In [22]:
ssce.date.isin(s2.date).all()

True

Excellent. I see no reason to pay attention to **Sweep Samples Cereals Edited** or **Sweep Samples Cereals** anymore, except for the possibility that the actual sample data differs. For the time being, I'll ignore that possibility and move ahead. 

#### Conclusion

Optimal candidate regarding these two sheets:

- **Sheet2**

### **Sheet2** vs **leafhoppers 2016 cereal sweeps**

In [23]:
sheets_to_compare = {
    name: sheets[name]
    for name in (
        'leafhoppers 2016 cereal sweeps',
        'Sheet2',
    )
}
lh2016, s2 = sheets_to_compare.values()
sheet_names = sheets_to_compare.keys()

#### Columns

In [24]:
compare_columns(sheets_to_compare)

,leafhoppers 2016 cereal sweeps,Sheet2
( damsel bug)nabis_americoferus_adult,✖️,✅
1st_instar_ega,✖️,✅
1st_instar_macrosteles,✅,✅
2nd_instar_ega,✖️,✅
2nd_instar_macrosteles,✅,✅
3rd_instar_ega,✖️,✅
3rd_instar_ega_pre-alate,✖️,✅
3rd_instar_macrosteles,✅,✅
4th_instar_macrosteles,✅,✅
4th_instar_pre-alate,✖️,✅


Clearly, **leafhoppers 2016 cereal sweeps** is focused on leafhoppers. Because the object of this analysis is to compare aphid numbers, I don't see the relevance of this leafhopper data.

#### Conclusion

Given that the leafhopper data isn't pertinent, **Sheet2** remains the best candidate.

### Sweeps: Overall Best Data Source

After all analysis, **Sheet2** appears to be the purest, most relevant base for comparison of "sweep" sample data to that of "tiller" count data.

## Tiller Head Counts

### Candidates

Only two worksheets in the source Excel workbook seem relevant to my need for "tiller head" samples:

- **Head Counts**
- **Head Counts Edited**

### **Head Counts** vs **Head Counts Edited**

In [25]:
sheets_to_compare = {
    name: sheets[name]
    for name in (
        'Head Counts',
        'Head Counts Edited',
    )
}
hc, hce = sheets_to_compare.values()
sheet_names = sheets_to_compare.keys()

#### Columns

In [26]:
compare_columns(sheets_to_compare)

,Head Counts,Head Counts Edited
aphid_mummies_blk,✅,✅
aphid_mummies_brown,✅,✅
aphids_total,✅,✖️
bco/head,✖️,✅
bco_alate,✅,✖️
bco_apt,✅,✖️
bco_head,✅,✖️
bco_leaf,✅,✖️
bco_total,✅,✅
comments,✅,✅


The main difference here seems to be the aggregation in the "edited" sheet, indicated by the columns named ***EGA/head***, ***EGA_total***, etc. For the purpose of achieving the project objective, the columns representing sums of values from other columns would likely be redundant because I intend to use the original values only.

#### Date Range

Now, compare for completeness:

In [27]:
def len_unique(pandas_object):
    return len(pandas_object.unique())

In [28]:
pandas.DataFrame(
    {
        name: frame.date
        for name, frame in sheets_to_compare.items()
    }
).apply([
    pandas.Series.max,
    pandas.Series.min,
    len_unique,
    len,
])

,Head Counts,Head Counts Edited
max,2016-08-15 00:00:00,2016-08-15 00:00:00
min,2016-07-11 00:00:00,2016-07-11 00:00:00
len_unique,20,20
len,216,216


These may be using the exact same set of dates. I'll confirm:

In [29]:
hc.date.index.isin(hce.date.index).all()

True

Identical date and time for the index of each, so the ***date*** column shows no basis for choosing one over the other.

### Head Counts: Overall Best Data Source

Since the only difference between these two frames is the unwanted extra columns, the simplest sheet wins this.

- **Head Counts**

# Align Selected Data Sets

The names and corresponding data frames, from the worksheets in the source workbook document (Excel):

In [30]:
sheets_to_compare = {
    'Head Counts': hc,
    'Sheet2': s2,
}
sheet_names = sheets_to_compare.keys()

## Compare Columns

Comparing alphabetically sorted column names between our two primary data sets:

In [31]:
pandas.concat(
    (
        pandas.DataFrame(
            data={
                's2 only': s2.columns.difference(hc.columns),
            }
        ),
        pandas.DataFrame(
            data={
                'hc only': hc.columns.difference(s2.columns),
            }
        ),
        pandas.DataFrame(
            data={
                'common': s2.columns.intersection(hc.columns),
            }
        ),
    ),
    sort=False,
    axis='columns',
).fillna('--')

,s2 only,hc only,common
0,( damsel bug)nabis_americoferus_adult,aphid_mummies_blk,date
1,1st_instar_ega,aphid_mummies_brown,site
2,1st_instar_macrosteles,aphids_total,crop
3,2nd_instar_ega,bco_alate,--
4,2nd_instar_macrosteles,bco_apt,--
5,3rd_instar_ega,bco_head,--
6,3rd_instar_ega_pre-alate,bco_leaf,--
7,3rd_instar_macrosteles,bco_total,--
8,4th_instar_macrosteles,comments,--
9,4th_instar_pre-alate,ega_alate,--


That's quite a large difference in columns for these sets. Because my target sample set is only aphids, I can first drop the unimportant columns, then normalize all remaining labels.

Before dropping any columns, I need to identify any remaining columns with corrolative properties, such as time and place labels which can serve as indices.

### Lookup Columns (Record Index)

Reading through the list of remaining, non-aphid related columns, I see some that don't mention any organism by name. These columns may be useful for indexing, which is crucial to aligning the two data sources.

In [32]:
non_organism_column_names = pandas.Series(data=(
    'collection_date',
    'comments',  # may contain identifying information
    'crop',
    'date',
    'date_by_week',  # redundant date coding
    'distance(m)',
    'field',
    'field_name',
    'id',
    'julian_date',  # redundant date coding
    'number of samples',  # unnecessary (used for sums, which are redundant)
    'province',
    'sample_by_week',  # 
    'site',
    'zadoks_stage',
))

Here are the names of the matching columns from each frame:

In [33]:
non_organism_columns_common = pandas.DataFrame(
    {
        name: dict(zip(non_organism_column_names,
                       non_organism_column_names.isin(frame.columns)))
        for name, frame
        in zip(sheet_names, (hc, s2))
    }
)
emojify_boolean(non_organism_columns_common)

,Head Counts,Sheet2
collection_date,✖️,✖️
comments,✅,✖️
crop,✅,✅
date,✅,✅
date_by_week,✖️,✅
distance(m),✖️,✅
field,✅,✖️
field_name,✖️,✅
id,✖️,✅
julian_date,✖️,✅


## Align Columns

Our next goal is to determine which columns are in common and ensure they're of the same data type, so we can concatenate the frames.

In [34]:
non_organism_columns_common.index[
    non_organism_columns_common.all(axis='columns')
].tolist()

['crop', 'date', 'site']

Those columns alone are probably enough to align the data.

I'll need to clean up `crop` and `site`, but `date` and `collection_date` are already well formed.

These columns warrant examination as well:

- collection_date
- date
- distance(m)
- field
- field_name
- number of samples

### Visualize Leading & Trailing Whitespace

Having peeked ahead a bit, I've noticed irregular use of space characters in some of the non-numeric values within the `DataFrame` objects. I could just strip all leading and trailing spaces, but I might overlook something along the way, and it's not a good habit to attempt to clean text too early in the course of analysis.

If I could see the spaces more easily, I could be acutely aware of them at any string's beginning and ending. To visualize these invisible characters, I'll compose a function that replaces the space character with something more visible. Displaying values this way makes trailing or leading whitespace obvious, without overwriting any data.

In [35]:
def show_spaces(x):
    return x.str.replace(
        ' ', '⬜️'
    )

### Date

The columns containing date information were normalized in an [earlier stage] of my analysis. Those columns are ready to align in combination with other unique index labels.

[earlier stage]: #Convert-Date-&-Time-Format

### Crop

Here are the `crop` field values for both data frames:

In [37]:
crops = pandas.concat(
    (
        frame.crop.apply(str)
        for frame in sheets_to_compare.values()
    ),
    keys=sheet_names,
    sort=True,
).drop_duplicates().reset_index(drop=True).sort_values()

show_spaces(crops)

3                   0
2              Barley
9            Barley⬜️
16                Oat
15              Oat⬜️
6                Oats
1               Tanzy
0               Wheat
8             Wheat⬜️
10      Winter⬜️Wheat
14    Winter⬜️Wheat⬜️
13        WinterWheat
12       Winter_wheat
11             barley
5                 nan
7            unlisted
4               wheat
Name: crop, dtype: object

Clearly, there are some variations that should be corrected, in both data frames.

- whitespace
- letter case
- word separation

I'll write a function that transforms any given "crop" value into a uniform representation of the crop it's intended to represent.

In [38]:
def normalize_str(value, separator=' '):
    if value is pandas.np.nan:
        return value

    str_value = str(value)

    # Add separators between words, title case
    is_mixed_case = str_value.upper() != str_value.lower() and not (str_value.islower() or str_value.isupper())
    if is_mixed_case:
        word_index = [
            index for index, char in enumerate(str_value)
            if char.isupper()
        ] + [None]  # For index slicing
        if word_index:
            words = [
                str_value[word_index[i]:word_index[i + 1]].strip()
                for i in range(len(word_index) - 1)
            ]
            str_value = separator.join(words)

    transformed = re.compile(r'[^a-zA-Z0-9]').sub(separator, str(str_value).lower())

    # De-pluralize
    if transformed.endswith('oats'):
        transformed = transformed[:-1]

    return transformed

Previewing the results, with the square brackets added again:

In [39]:
pandas.concat(
    (
        crops,
        crops.apply(normalize_str),
    ),
    keys=("Before", "After"),
    axis='columns',
).sort_values('After').apply(show_spaces)

,Before,After
3,0,0
2,Barley,barley
9,Barley⬜️,barley
11,barley,barley
5,nan,nan
16,Oat,oat
15,Oat⬜️,oat
6,Oats,oat
1,Tanzy,tanzy
7,unlisted,unlisted


There are some concerning names, such as "nan", "unlisted", and "0", but the renaming result looks good. I'll apply the changes:

In [40]:
for frame in (hc, s2):
    frame.crop = frame.crop.apply(normalize_str)

### Site

In [41]:
site_values = (
    pandas.concat(
        (frame[['site']] for frame in (hc, s2)),
        keys=sheet_names,
        names=['Sheet Name', 'index',],
    )
    .drop_duplicates()
    .sort_values('site')
)
show_spaces(site_values.site)

Sheet Name   index
Head Counts  8               Alberta
             142              Alvena
Sheet2       174              Clavet
Head Counts  141        Indian⬜️Head
             128              Kernan
Sheet2       193            Kernan⬜️
Head Counts  109           Llewellyn
             53             Manitoba
             103      Meadow⬜️Lake⬜️
             92          Meadow_Lake
             96              Melfort
Sheet2       363           Melfort⬜️
Head Counts  54              Outlook
             75            Outlook⬜️
             10                  SEF
             27                SEF⬜️
             4                 Wakaw
             0         Yellow⬜️Creek
Sheet2       626       Yellow⬜️creek
Head Counts  2          Yellow_Creek
Sheet2       632         Yellowcreek
Name: site, dtype: object

This is going to need some normalization. I'll write a string reducer function to transform any given value into a uniformly reducible representation by stripping insignificant characters and letter case. From that, I can build a hash table that maps to preferred representations, and apply that mapping to the values in a renaming operation.

#### Reduce Label Value

In order to match badly formed labels with their normal representation, I need a string function similar to a hash, so the values in the data frame can be used to look up the preferred, normal form.

- convert numeric values to text
- strip non alphanumeric characters
- convert alphabetic characters to lower case

In [42]:
def hash_like(value):
    return re.compile(r"[^a-z0-9]").sub('', str(value).lower())

Applying this to all frames:

In [43]:
for frame in (hc, s2) + (site_values, ):
    frame['site_index'] = frame.site.apply(hash_like)
    frame.set_index('site_index', append=True, inplace=True)

#### Nominal Values

If I had a larger data set, I could automatically find the most frequently used representation for any given reduced value ("hash"); I would use the `mode` function. Unfortunately, the groups are far too small and the values too varied:

In [44]:
(
    site_values
    .reset_index(level=['Sheet Name', 'index'], drop=True)
    .loc[:, ['site']]
    .apply(show_spaces)
)

,site
site_index,
alberta,Alberta
alvena,Alvena
clavet,Clavet
indianhead,Indian⬜️Head
kernan,Kernan
kernan,Kernan⬜️
llewellyn,Llewellyn
manitoba,Manitoba
meadowlake,Meadow⬜️Lake⬜️


Even if I strip the leading and trailing whitespace, I would still end up with ambiguous candidate selections. The work required to make a function that would know to how and when to convert labels like `Yellowcreek` to `Yellow Creek` would be unreasonable given the scope of this project, so automation might not be the best choice for choosing normal forms.

I'll make the preferred identifier list manually, then apply the changes automatically. Here it is as a data frame with the appropriately reduced version of each label as the index:

In [45]:
preferred_site_id = pandas.Series(
    name='site',
    data={hash_like(item): item
          for item in [
              'Alvena',
              'Clavet',
              'Indian Head',
              'Kernan',
              'Llewellyn',
              'Meadow Lake',
              'Melfort',
              'Outlook',
              'SEF',
              'Wakaw',
              'Yellow Creek',
          ]},
)
preferred_site_id.index.set_names(['site_index'], inplace=True)
preferred_site_id.to_frame().T

site_index,alvena,clavet,indianhead,kernan,llewellyn,meadowlake,melfort,outlook,sef,wakaw,yellowcreek
site,Alvena,Clavet,Indian Head,Kernan,Llewellyn,Meadow Lake,Melfort,Outlook,SEF,Wakaw,Yellow Creek


With this list, I can compare the reduced ("hashed") values to the ones in the actual data and apply the preferred name where it matches.

Everything that isn't in the preferred site name list:

In [46]:
site_values[
    ~ site_values.index.get_level_values('site_index')
    .isin(preferred_site_id.index)
]

site
Sheet Name  index site_index          
Head Counts 8     alberta      Alberta
            53    manitoba    Manitoba

These outliers are from the unmatchable records I mentioned upon my first look at the [unique site] labels. I can move on without doing anything further on these.

[unique site]: #Site

Report on which records would be changed if I relabel ***site***:

In [47]:
pandas.concat(
    (
        site_values,
        preferred_site_id.to_frame().combine_first(site_values),
    ),
    keys=['Before', 'After'],
    axis='columns',
).reset_index('site_index', drop=True).apply(show_spaces)

Before          After
                             site           site
Sheet Name  index                               
Head Counts 8             Alberta        Alberta
            142            Alvena         Alvena
Sheet2      174            Clavet         Clavet
Head Counts 141      Indian⬜️Head   Indian⬜️Head
            128            Kernan         Kernan
Sheet2      193          Kernan⬜️         Kernan
Head Counts 109         Llewellyn      Llewellyn
            53           Manitoba       Manitoba
            103    Meadow⬜️Lake⬜️   Meadow⬜️Lake
            92        Meadow_Lake   Meadow⬜️Lake
            96            Melfort        Melfort
Sheet2      363         Melfort⬜️        Melfort
Head Counts 54            Outlook        Outlook
            75          Outlook⬜️        Outlook
            10                SEF            SEF
            27              SEF⬜️            SEF
            4               Wakaw          Wakaw
            0       Yellow⬜️Creek  Yellow⬜️Creek
Sheet2      626     Yellow⬜️creek  Yellow⬜️Creek
Head Counts 2        Yellow_Creek  Yellow⬜️Creek
Sheet2      632       Yellowcreek  Yellow⬜️Creek

This looks good to me. I'll apply the names:

In [48]:
for frame in (hc, s2):
    frame.loc[:, 'site'] = (
        preferred_site_id.to_frame()
        .combine_first(frame)
        .loc[:, 'site']
    )
    frame.reset_index(
        level='site_index',
        drop=True,
        inplace=True,
    )

I'll review the label values for ***site*** in both data frames, to see the result of those changes:

In [49]:
show_spaces(
    pandas.concat(
        (frame[['site']] for frame in (hc, s2)),
    )
    .loc[:, 'site']
    .reset_index(drop=True)
    .sort_values()
    .drop_duplicates()
)

82           Alberta
228           Alvena
392           Clavet
141     Indian⬜️Head
474           Kernan
561        Llewellyn
84          Manitoba
105     Meadow⬜️Lake
627          Melfort
59           Outlook
791              SEF
824            Wakaw
863    Yellow⬜️Creek
Name: site, dtype: object

### Field

The columns **hc**.***field*** and **s2**.***field_name*** seem to relate to ***site*** and ***crop***. Before I address the values of the columns, I'll rename **s2**.***field_name*** for consistency.

In [50]:
for frame in (hc, s2):
    frame.rename(
        columns={'field_name': 'field'},
        inplace=True,
    )

Now, regarding the values, I suspect redundancy in ***field***. Here's why I feel that way—look at some of the values from both data sets:

In [51]:
pandas.options.display.max_rows = 20

column_names = ['crop', 'site', 'field', 'date']
(
    pandas.concat(
        (
            hc[column_names],
            s2[column_names],
        ),
        keys=sheet_names,
        names=['worksheet', 'index',],
    )
    .sort_values(
        by=['crop', 'site', 'date', 'field'],
    )
    .set_index(
        keys=['crop', 'site'],
    )
    .drop_duplicates()
    .apply(
        {
            'field': show_spaces,
            'date': lambda x: x,
        }
    )
)

field       date
crop         site                                             
0            Alberta         SW25-11-11-W4⬜️CM11687 2016-08-02
barley       Alvena                      AlveBarley 2016-06-10
             Alvena                    AlvenaBarley 2016-06-23
             Alvena                    AlvenaBarley 2016-06-24
             Alvena                    AlvenaBarley 2016-07-08
             Alvena                    AlvenaBarley 2016-07-15
             Alvena                    AlvenaBarley 2016-07-17
             Alvena                 Alvena⬜️Barley1 2016-07-22
             Alvena                    AlvenaBarley 2016-07-22
             Alvena                    AlvenaBarley 2016-07-28
...                                             ...        ...
wheat        Yellow Creek               YellowWheat 2017-07-08
winter wheat Alvena                     KernanWheat 2016-07-15
             Kernan               KernanWinterWheat 2016-06-28
             Kernan        Kernan⬜️Winter⬜️Wheat⬜️1 2016-07-21
             Kernan                     KernanWheat 2016-07-28
             Llewellyn        Llewellyn_winterwheat 2016-08-05
NaN          Alberta               NW-09-06-15-W4⬜️ 2016-07-27
             Alberta        NE-09-06-15-W4⬜️CM11686 2016-08-02
             Alberta         NE33-04-09-W4⬜️CM11685 2016-08-02
             Alberta         NE33-04-09-W4⬜️CM11690 2016-08-08

[211 rows x 2 columns]

It's clear to my eyes that in some cases, ***site*** and ***crop*** have been concatenated together to produce the value in ***field***.

- ***site*** + ***crop*** = ***field***

Primary data:

- ***site***
- ***crop***

Mixed data:

- ***field***

Therefore, in samples corresponding to purely derivative ***field*** values, I can safely disregard those derived columns and rely on the the more normalized ***crop*** and ***site*** for indexing. That is to say I think it's most beneficial to use ***crop*** and ***site*** rather than ***field*** whenever possible.

The only complication is the numeric suffix present in some values of ***field***, which delineates areas of the site at which the samples were observed. The numeric values are unique per date-crop-place combination but not unique per site-crop combination, so I'll have to treat it as a separate column that represents an additional dimension to our independent variables.

Extracting the unique pairs of dates and the corresponding numerals from the processed ***field*** value for that date:

In [52]:
pandas.options.display.max_rows = 40

(
    Out[63]
    .reset_index()
    .apply(
        dict(
            tuple(
                dict.fromkeys(
                    (
                        'date',
                        'site',
                        'crop',
                    ),
                    lambda x: x,
                ).items() 
            ) + tuple(
                {
                    'field': lambda x: x.str.extract(
                        pat=r'(?P<text>\D*)(?P<number>\d*)',
                    ),
                }.items(),
            ),
        )
    )
    .drop_duplicates(
        subset=[
            ('date', 'date'),
            ('field', 'number'),
        ],
    )
    .dropna(
        subset=[
            ('date', 'date'),
            ('field', 'number'),  # @todo: needed?
        ],
    )
)


KeyError: 63

Based on this look at the information, I'd like to throw away the remaining text portion after extractinng the numbers when I apply this to my data set.

When it comes to actually using the number portion of the ***field*** column rather than merely displaying it here in the notebook, it's simpler just to assign a new `Series` to the `DataFrame` than it is to non-destructively express the intended outcome.

In [0]:
for sheet in (hc, s2):
    sheet.field = (
        sheet.field
        .str.extract(pat=r'(?P<text>\D*)(?P<number>\d*)')
        .loc[:, 'number']
        .apply(pandas.to_numeric, downcast='integer')
    )

Just a reminder that the samples corresponding to ***site*** values `Alberta` and `Manitoba` use dates which don't overlap with the dates in the other `DataFrame`, so the fact that the ***field*** values for these records have a different pattern and are therefore mangled is completely fine for this project—they'll be entirely dropped in an upcoming phase of my work.

### Number of Samples & Distance

The ***number of samples*** field is only present in **Sheet2**. I want to know a little bit about the values:

In [0]:
s2['number of samples'].value_counts(dropna=False)

There are many missing values in this column. I presume the number of samples relates to the "combined" values, but I need to confirm:

In [0]:
s2[['distance(m)', 'number of samples']].dropna(how='all').drop_duplicates()

This confirms that where the ***number of samples*** is not indicated, the corresponding ***distance(m)*** value is numeric. The converse is also true: non-numeric ***distance(m)*** corresponds with positive values in ***number of samples***. That's consistent with my expectations for the appearance of labels on previously aggregated data.

### Identify Columns With Aphid Sample Data

The first step in normalizing column names for aphid samples is identifying the set of columns that mention aphids by name. For better comparison, I'll filter for columns referring to aphids.

Here are some terms I know to be indicative of a relationship to aphid samples:

* aphid
* ega
* bco
* greenbug

Not aphid related:

- aphidencyrtus
- aphidiius

Categorized as natural enemy:

- aphid_mummies

Analytical artifacts:

- total

Following these guidelines, the relevant columns are:

**English grain aphid**

* 1st_instar_ega
* 2nd_instar_ega
* 3rd_instar_ega
* 3rd_instar_ega_pre-alate
* ega alate
* ega_alate
* ega_apt
* ega_grn
* ega_head
* ega_leaf
* ega_red
* sitobion_avenae_ega_green (wingless)
* sitobion_avenae_ega_red

This is ambiguous, but it's probably referring to EGA:

* 4th_instar_pre-alate

**Bird cherry oat aphid**

* bco_alate
* bco_apt
* bco_head
* bco_leaf
* bird_cherry_oat_aphid

**Greenbug aphid**

* greenbug_alate
* greenbug_apt
* greenbug_aphid

**Pea aphid**

* pea aphids

The pea aphid data is only in **s2**, so it can be disregarded.

In [0]:
aphid_column_names = [
    '1st_instar_ega',
    '2nd_instar_ega',
    '3rd_instar_ega',
    '3rd_instar_ega_pre-alate',
    'ega alate',
    'ega_alate',
    'ega_apt',
    'ega_grn',
    'ega_head',
    'ega_leaf',
    'ega_red',
    'sitobion_avenae_ega_green (wingless)',
    'sitobion_avenae_ega_red',
    '4th_instar_pre-alate',
    'bco_alate',
    'bco_apt',
    'bco_head',
    'bco_leaf',
    'bird_cherry_oat_aphid',
    'greenbug_alate',
    'greenbug_apt',
    'greenbug_aphid',
]

In [0]:
visualize_boolean(
    pandas.DataFrame(
        data={
            name: sheet.columns.to_series(name=name).loc[
                sheet.columns.isin(aphid_column_names)
            ]
            for name, sheet in sheets_to_compare.items()
        },
    )
)

### Observation Data

Now that the data frame indices I built from the ***date***, ***site***, ***field*** and ***crop*** columns align, I need to determine which observations were recorded in both data frames.

Some column names are nearly identical to ones in the opposite `DataFrame`, so I'll need to normalize the names if I expect to align the corresponding sample values.

#### Correlate Aphid Columns by Aphid Type

I'll be looking for aphids that were measured in both sources, and I'll ultimately want only one column from each `DataFrame` for each type of aphid that is present in both sources. Some aphid types aren't present in both `DataFrames`, so I can safely ignore those.

Knowing this, I can proceed to drop all other aphid columns and focus on the two usable types:

- English grain aphid
- Greenbug

##### Reduce Phenotypes

###### @todo

- [ ] match (between sheets) columns like 'alate' rather than overall sums!

Greenbug:

- sum of hc:greenbug_.+
- s2:greenbug_aphid

EGA:

Alate:

- hc:ega_alate
- s2:ega alate

All:

- s2:sum all
- hc:sum of any of [apt, alate], [grn, red], [head, leaf]

I already anticipate another problem to solve: the labels for aphids indicate multiple variations and probably multiple representations of the same aphid species count in the same `DataFrame`. For example, English grain aphids from **Head Counts** are winged & wingless, red & green, but there are also "head" & "leaf" counts. Compare that with the data labels in **Sheet2**, where there are larva & adults (instar), pre-alate & alate, red & green (wingedness not specified), wingless green & wingless with no colour specified—but no ***ega_leaf*** or ***ega_head***.

I'll have to figure out which columns to combine and which to ignore. I'll begin with the presumption that ***ega_head*** & ***ega_leaf*** should produce the same total as the pairings of ***ega_apt*** & ***ega_alate*** or ***ega_red*** & ***ega_grn***. If I can make sense of those columns in **Head Counts**, I'll proceed to examine the value totals in **Sheet2** the same way.

Greenbugs will be much simpler, given that I'll be cross referencing one simple column, ***greenbug_aphid*** with only two, mutually exclusive columns from the other worksheet, ***greenbug_alate*** & ***greenbug_apt***. The sum of counts for the two greenbug phenotypes of **Sheet2** should correspond fully with the solitary counterpart in **Head Counts**.

###### English Grain Aphid Counts, **Head Counts**

I'm considering which columns to use, and I'm thinking of using the sum of ***ega_head*** and ***ega_leaf***.
Given all logical pairings of EGA columns in **Head Counts**, I expect the sums to be identical amongst all three pairs for each of the sampling dates. If I add a column showing the standard deviation for each row, I should see zeros all down that column.

In [0]:
hc_ega_sums = pandas.concat(
    (
        (hc['ega_head'] + hc['ega_leaf']),
        (hc['ega_apt'] + hc['ega_alate']),
        (hc['ega_red'] + hc['ega_grn']),
    ),
    axis='columns',
    keys=['head + leaf', 'apt + alate', 'reg + grn',],
)
hc_ega_sums['std'] = hc_ega_sums.std(
    axis='columns',
)

pandas.options.display.max_rows = 10

hc_ega_sums.sort_values('std', ascending=False,)

Looks like there are many invalid counts. I need to see the proportion of outliers to the entire data set:

In [0]:
len(hc_ega_sums[hc_ega_sums['std'] > 0]) / len(hc_ega_sums)

Less than 20% of the counts have discrepencies. I feel it's probably safe to drop all rows with these dates as outliers.

##### @todo:

- [ ] drop rows with non-zero standard deviation
- [ ] apply normalized aphid names

### Remove Unnecessary Columns

Aligning the records will be simplest if there are no extraneous columns. Ideally, I should be able to align the rows and start using the joined data set, without selecting columns. What non-aphid, non-lookup columns remain?

##### @todo:

- [ ] identify remaining columns
- [ ] drop other columns

### Compare Group Sums to Pre-existing "Combined"

I didn't expect that the data set from **Sheet2** would have sums (presumably from groupings of ***distance(m)*** values), yet also some non-aggregated values. Normally, these wouldn't be mixed, because they represent different dimensional orders. Because this data set's dimensional order is inconsistent, there is probably redundancy in the total information available, and possibly contradictions.

Any redundancy, whether contradictory or not, would affect calculation for the intended [objective], unless there's exactly one record for each space and time combination—that is to say, if there are discrete values as well as previously "combined" values for the same point along the index, I'll have to avoid including the pre-calculated sum when aggregating my own sums, otherwise the resulting totals will be doubled.

[objective]: #Objective

I'll separate discrete and aggregated sets, then compare my sums of discrete sample values to pre-existing aggregations.

Before I can compare

## Align Rows

##### @todo:

- [ ] mark bad dates, drop rows when aligning rows